In [1]:
import requests
import bs4 as BeautifulSoup
import pandas as pd
import re

In [2]:
def get_page(url):
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}
)
    
    if not response.ok:
        print('Server responded:', response.status_code)
    else:
        soup = BeautifulSoup.BeautifulSoup(response.text, 'lxml')
    return soup

In [4]:



def get_detail_data(soup):
    """
    Scraping info from listing: title, price, number sold, shipping_type, condition, condition description.
    """
    try:
        title = soup.body.find('div', id='LeftSummaryPanel').find('span', class_='ux-textspans ux-textspans--BOLD').text.lower()
    except:
        title = None

    try:
        try:
            price = soup.body.find('span', id="prcIsum").text
            price = float(re.findall(r"[-+]?\d*\.\d+|\d+", price)[0])
        except:
            price = soup.body.find('span', id="mm-saleDscPrc").text
            price = float(re.findall(r"[-+]?\d*\.\d+|\d+", price)[0])
    except:
        price = None



    try:
        num_sold = soup.body.find('div', class_='vi-quantity-wrapper').find('span', class_='vi-qtyS-hot-red').a.text
    except:
        num_sold = None

    try:
        shipping_type = soup.body.find('div', class_='ux-labels-values__values col-9').find('span', class_='ux-textspans ux-textspans--POSITIVE ux-textspans--BOLD').text.lower()
    except:
        shipping_type = None

    try:
        condition = soup.body.find('div', id='mainContent').find_all('span', class_='ux-textspans')[1].text.lower()
    except:
        condition = None

    try:
        condition_description = soup.body.find('div', class_='ux-layout-section__item ux-layout-section__item-background ux-layout-section__item--table-view').find_all('div', 'ux-layout-section__row')[1].find('div', class_='ux-labels-values__values').span.text.lower()
    except:
        condition_description = None
     
    try:
        num_reviews = int(soup.body.find('div', class_='ux-seller-section__item--seller').find('span', class_='ux-textspans ux-textspans--PSEUDOLINK').text)
    except:
        num_reviews = None
    
    try:
        pos_feedback_pct = soup.body.find('div', class_='ux-seller-section__content').find_all('div', class_='ux-seller-section__item')[1].span.text
        pos_feedback_pct = float(re.findall(r"[-+]?\d*\.\d+|\d+", pos_feedback_pct)[0])

    except:
        pos_feedback_pct = None
    
    try:
        return_days = soup.body.find('div', class_='vim x-returns-minview').find('div', class_='ux-layout-section__item').find('div', class_='ux-layout-section__row').find_all('span')[1].text
    except:
        return_days = None

    
    # Not working well yet
    try:
        brand = soup.body.find('div', class_='ux-layout-section ux-layout-section--features').find('div', class_="ux-layout-section__row").find_all('div', class_='ux-labels-values__values')[0].find('span', class_="ux-textspans").text
    except:
        brand = None

    try:
        model = soup.body.find('div', class_='ux-layout-section ux-layout-section--features').find('div', class_="ux-layout-section__row").find_all('div', class_='ux-labels-values__values')[1].find('span', class_="ux-textspans").text
    except:
        model = None

    # Testing out now
   

    


    data = {
        'title': title,
        'price': price,
        'num_sold': num_sold,
        'shipping_type': shipping_type,
        'condition': condition,
        'condition_description': condition_description,
        'num_reviews': num_reviews,
        'pos_feedback_pct': pos_feedback_pct,
        'return_days':return_days,
        # 'brand': brand,
        # 'model':model,

    }
    return data





In [21]:
url = 'https://www.ebay.com/itm/144549824735?epid=14026729707&hash=item21a7d748df:g:secAAOSwMWpiV0Iq'
test_soup = get_page(url)
result = get_detail_data(get_page(url))

In [91]:
test_soup.body.find_all('div')[38].text

'Apple iPhone XS - 64GB - (Unlocked) A1920'

In [92]:
url = 'https://www.ebay.com/itm/255546416585?epid=8039807695&hash=item3b7fc0ddc9:g:MBoAAOSwSrtiiUi9'
test_soup = get_page(url)
result = get_detail_data(get_page(url))

In [189]:
test_soup.body.find('div', class_='vim x-returns-minview').find('div', class_='ux-layout-section__item').find('div', class_='ux-layout-section__row').find_all('span')[1].text

'30 days returns'

In [190]:
test_soup.body.find('div', class_='vim x-returns-minview').find('div', class_='ux-layout-section__item').find('div', class_='ux-layout-section__row').find_all('span')[3].text

'Seller pays for return shipping'

In [191]:
test_soup.body.find('div', class_='ux-layout-section ux-layout-section--features').find('div', class_="ux-layout-section__row").find_all('div', class_='ux-labels-values__values')[1].find('span', class_="ux-textspans").text

'iPhone 7'

In [192]:
main_page_url = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2334524.m570.l1313&_nkw=iphone&_pgn=1'

In [193]:
def get_index_data(soup):
    try:
        links = soup.find_all('a', class_='s-item__link')
    except:
        links = []
    urls = [item.get('href') for item in links]
    
    return urls

In [194]:
pages = []
for i in range(1,100):
    pages.append(get_index_data(get_page(f'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2334524.m570.l1313&_nkw=iphone&_pgn={i}')))

In [195]:
pages[4]

['https://ebay.com/itm/123456?hash=item28caef0a3a:g:E3kAAOSwlGJiMikD&amdata=enc%3AAQAHAAAAsJoWXGf0hxNZspTmhb8%2FTJCCurAWCHuXJ2Xi3S9cwXL6BX04zSEiVaDMCvsUbApftgXEAHGJU1ZGugZO%2FnW1U7Gb6vgoL%2BmXlqCbLkwoZfF3AUAK8YvJ5B4%2BnhFA7ID4dxpYs4jjExEnN5SR2g1mQe7QtLkmGt%2FZ%2FbH2W62cXPuKbf550ExbnBPO2QJyZTXYCuw5KVkMdFMDuoB4p3FwJKcSPzez5kyQyVjyiIq6PB2q%7Ctkp%3ABlBMULq7kqyXYA',
 'https://www.ebay.com/itm/174911456075?epid=13034214288&hash=item28b989034b:g:UlgAAOSwthJhE782&amdata=enc%3AAQAHAAAA4HaUkdPbkM%2FP4J%2BjLOhsFoExheK%2BV2QJqfLWEprH2Hd73fqWKVEFnNyEhNyyw%2F0oPpoM4VDYHoG9Imj0SLPOEKYKOj3Ndv1gOW0gliJpEz7urdchnuHbmusjm%2B3w9fQ807FKxNG9oSoDNmaB98KpUGdA3I5XQHmDHc%2BCpsVHsM%2FRwFKeNPNvfR1jHXOe0cy1ybuWQeppNP%2F%2BsLr1aXcPT95ukoLIcd%2Bx7nxcFniHLwiyLCWhdyVGbWMUFQRqVWOwUpGr7vLVGKP1WU3T4%2Bd50CArXuC9QFwhU9rnauVBBbQx%7Ctkp%3ABFBM4vugqZxg',
 'https://www.ebay.com/itm/353644775166?epid=24049349860&hash=item5256df06fe:g:YKUAAOSwfNJhKAfK&amdata=enc%3AAQAHAAAA0A0R8EOzWMoiUPzbCty9yBNcl24OfODprS%2BXBDN%2BQU3wPOflVrR2

In [196]:
data_list = []

for list in pages:
        for link in list:
            try:
                data = get_detail_data(get_page(link))
                data_list.append(data)
            except:
                pass
    


Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server responded: 404
Server res

In [197]:
iphones_df = pd.DataFrame(data_list)
iphones_df

,title,price,num_sold,shipping_type,condition,condition_description,num_reviews,pos_feedback_pct,return_days
0,apple iphone 7 plus (excellent condition) fact...,237.99,19 sold,free 4 day shipping,used,“fully functional in excellent condition. our ...,30470.0,99.9,30 days returns
1,apple iphone 8 64gb gsm factory unlocked gray ...,149.99,"2,313 sold",free 2 day shipping,good - refurbished,“good overall condition. may have some minor s...,2703.0,97.2,30 days returns
2,apple iphone 8 64gb gsm factory unlocked smart...,149.99,"2,419 sold",free 3 day shipping,used,“overall good condition will have marks and sc...,13103.0,96.3,30 days returns
3,apple iphone 7 32gb gsm factory unlocked 4g lt...,107.99,"6,010 sold",free 3 day shipping,used,“overall good working condition and will have ...,13103.0,96.3,30 days returns
4,apple iphone x 64gb factory unlocked phone - v...,209.95,"22,036 sold",free 4 day shipping,very good - refurbished,"“this is a b+ stock item, meaning unit is in v...",332288.0,98.4,60 days returns
...,...,...,...,...,...,...,...,...,...
7440,apple iphone 6 16gb gsm unlocked a1549 smartph...,119.95,None,free 4 day shipping,used,“iphone is in excellent condition. comes with...,2116.0,100.0,30 days returns
7441,apple iphone 4 - 8gb - black (at&t) a1332 (gsm),8.60,None,None,for parts or not working,None,87.0,96.8,Seller does not accept returns
7442,apple iphone 6 plus 16gb silver a1522 (sprint ...,62.97,None,None,used,“fully working (minor issues)\nfully working w...,136775.0,98.8,30 days returns
7443,apple iphone 6 128gb unlocked -excellent condi...,99.00,None,None,used,“excellent”,884.0,100.0,30 days returns


In [198]:
iphones_df.isna().sum()

title                      15
price                     233
num_sold                 6369
shipping_type            4505
condition                  15
condition_description    3452
num_reviews                16
pos_feedback_pct          146
return_days                15
dtype: int64

In [201]:
iphones_df.to_csv('iphones_datafram.csv')